In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import sys
ancestor = os.path.abspath(os.path.join(os.pardir, os.pardir))
if ancestor not in sys.path:
    sys.path.append(ancestor)
    sys.path.append(os.pardir)
from classifier import get_Dx, pair_diagnoses, asd, adhd, anx
from CrossCat.functions import dependent_variables, mri_features

In [3]:
testing1_ASD = pd.read_csv('testing1_ASD.csv')

In [4]:
training1_ASD = pd.read_csv('training1_ASD.csv')

In [5]:
mri_df = pair_diagnoses(
    pd.DataFrame({
        "index": [
            d for d in os.listdir(
                os.path.join(
                    os.pardir,
                    "fMRI"
                )
            ) if d.startswith("N")
        ]
    }).set_index("index"),
    get_Dx()
).reset_index().rename(
    index=str,
    columns={
        "index": "EID"
    }
)

In [6]:
mri_subtypes = "mri_subtypes.csv"
mri_df.to_csv(
    mri_subtypes,
    index=False
)
dependent_variables(mri_subtypes)

In [7]:
mri_table = pair_diagnoses(
    pd.concat([mri_features(d) for d in os.listdir(
        os.path.join(
            os.pardir,
            "fMRI"
        )
    ) if d.startswith("N")]),
    get_Dx()
).reset_index().rename(
    index=str,
    columns={
        "index": "EID"
    }
)
mri_table["ADHD subtype"] = mri_table["Dx"].apply(
    lambda x: ", ".join([
        s for s in x if "ADHD" in s
    ])
)
mri_table["ASD"] = mri_table["Dx"].apply(
    lambda x: True if(
        len(
            ", ".join([
                s for s in x if "Autism" in s
            ])
        )
    ) else False
)

In [8]:
mri_table.to_csv(
    "all_MRI_features.csv",
    index=False
)
dependent_variables("all_MRI_features.csv")
mri_table = pd.read_csv("all_MRI_features.csv")

In [9]:
full_set = pd.merge(
    pd.concat([
        testing1_ASD.drop(
            [
                "train",
                "Dx2"
            ],
            axis=1
        ),
        training1_ASD.drop(
            [
                "train",
                "Dx2"
            ],
            axis=1
        )
    ]),
    mri_table,
    on=[
        "EID",
        "Dx",
        "ADHD subtype",
        "ASD"
    ],
    how="outer"
)

In [10]:
full_set = full_set.set_index("EID")

In [11]:
full_set.to_csv(
    "all_features.csv"
)

In [12]:
five_hundred = mri_table.copy()
for c in five_hundred.columns:
    if "_ants" in c:
        five_hundred.drop(
            c,
            axis=1,
            inplace=True
        )

In [13]:
five_hundred = five_hundred[
    [
        *[
            c for c in five_hundred.columns if (
                "vol" in c
            ) or (
                "area" in c
            ) or (
                "thickinthehead" in c
            ) or (
                (
                    "mean curvature" in c
                ) and (
                    "median" in c
                ) and (
                    "label" in c
                )
            )
        ],
        "EID",
        "Dx",
        "ADHD subtype",
        "ASD"
    ]
]

In [14]:
subset = pd.merge(
    pd.concat([
        testing1_ASD.drop(
            [
                "train",
                "Dx2"
            ],
            axis=1
        ),
        training1_ASD.drop(
            [
                "train",
                "Dx2"
            ],
            axis=1
        )
    ]),
    five_hundred,
    on=[
        "EID",
        "Dx",
        "ADHD subtype",
        "ASD"
    ],
    how="outer"
)

In [15]:
subset = subset.set_index("EID")
subset.to_csv("questions_and_mri_subset.csv")